In [1]:
import matplotlib.pyplot as plt
#from matplotlib import cm
#from matplotlib.ticker import LinearLocator
import numpy as np
import xarray as xr
import netCDF4 as nc
import math
import cftime
import numpy.polynomial.polynomial as pol

di = '/home/claire/Github-forked/MOM6-examples/ocean_only/seamount/pseudo_ISOMIP'

NHALO = 4
IC_FILE = 'MOM_IC.nc'
SHELF_IC_FILE = 'MOM_Shelf_IC.nc'
PROG_FILE = 'prog.nc'
ICE_SHELF_FILE = '/INPUT/shelfX.nc'
THICKNESS_FILE = '/INPUT/thickness.nc'

In [2]:
### new thickness file for THICKNESS_CONFIG="thickness_file"
### (depends on MOM6 output in IC_FILE, ie. use after a run)
def make_thickness_file(filename):
    
    ic = xr.open_dataset(di + '/' + IC_FILE)
    _, nz, ny, nx = ic.h.data.shape

    new_thick = nc.Dataset(filename, "w", format="NETCDF4")
    new_thick.createDimension("nz", nz)
    new_thick.createDimension("ny", ny)
    new_thick.createDimension("nx", nx)
    new_thick.createVariable("h","f8",("nz","ny","nx"))

    new_thick["h"][:,:,:] = ic.h.data
    new_thick.close()

In [3]:
### 3D depiction of topography and ice shelf
### depends on MOM6 output in IC_FILE and SHELF_IC_FILE
def display_config():
    
    ic = xr.open_dataset(di + "/"+ IC_FILE)
    ms = xr.open_dataset(di + "/" + SHELF_IC_FILE)
    #ms = xr.open_dataset(di + "/" + ICE_SHELF_FILE)
    #tp = xr.open_dataset(di + "/" + TOPO_FILE)

    LY, LX = ic.h.data[0].shape[1], ic.h.data[0].shape[2]
    YY= np.arange(0, LY, 1)
    XX = np.arange(0, LX, 1)
    XX, YY = np.meshgrid(XX, YY)

    fig, axes = plt.subplots(nrows = 1, ncols=2, figsize = (10,4), subplot_kw={"projection": "3d"})

    surf = axes[0].plot_surface(XX, YY, ms.h_shelf.data[0])
    #surf = axes[0].plot_surface(XX, YY, ms.thick.data)
    axes[0].set_title("Input ICE SHELF h")

    eta = ic.eta[0].data[0]
    bathyT = eta - np.sum(ic.h[0].data, axis=0)

    axes[1].plot_surface(XX, YY, eta)
    axes[1].plot_surface(XX, YY, bathyT)
    #topo2 = axes[1].plot_surface(XX, YY, -tp.depth.data)

    axes[1].set_title("Initial TOPOGRAPHY and ICE SHELF")

    ms.close()
    ic.close()

In [4]:
### New shelf: Take an ISOMIP cross-section, smooth it quadratically, and replicate non-uniformly into a dome
### <font color='green'>h_shelf is continuous in all directions</font>

### Bad ice draft

def make_shelf(filename):
    old_shelf = nc.Dataset(di + "/INPUT/Ocean1_3D.nc", "r")
    #print(old_shelf)

    b3 = old_shelf.variables["thick"][6,:]
    b5 = old_shelf.variables["area"][6,:]

    x=np.arange(240)
    #bb3=0.9*np.maximum(0.02*(x-250)**2-150,0) # max vel ~ 1e-5 after 5 days with 0.3 on line 23
    bb3=0.9*np.maximum(3e-7*(x-250)**4-17,0) # max vel ~ 1e-4 after 5 days with 0.8 on line 23

    new_shelf = nc.Dataset(filename, "w", format="NETCDF4")
    new_shelf.createDimension("ny", 40)
    new_shelf.createDimension("nx", 240)
    new_shelf.createVariable("thick","f8",("ny","nx"))
    new_shelf.createVariable("area","f8",("ny","nx"))

    for ii in range(40):
        new_shelf["area"][ii,:] = b5
        bb4 = bb3 +(19.5-ii)**2*0.8 # previously 0.3
        iip = abs(20-ii)
        for j in range(iip):
            bb4[164-j] = bb4[164-iip]*j/iip
        bb4[164:] = 0
        new_shelf["thick"][ii,:] = bb4

    new_shelf["thick"][:, 164:] = 0
    new_shelf["area"][:, 164:] = 0

    new_shelf.close()

In [5]:
### New shelf: Take an ISOMIP cross-section, smooth it quadratically, and replicate non-uniformly into a dome
### <font color='green'>h_shelf is continuous in all directions</font>

### Good ice draft

def make_shelf_2(filename):
    old_shelf = nc.Dataset(di + "/INPUT/Ocean1_3D.nc", "r")
    #print(old_shelf)

    b3 = old_shelf.variables["thick"][6,:]
    b5 = old_shelf.variables["area"][6,:]

    #fn = pol.Polynomial.fit([0,25,50,70,100,120,160],[1000,950,850,400,200,188,0],3)
    #fn = pol.Polynomial.fit([0,25,50,70,100,120,160],[1000,990,850,400,200,188,0],3)
    fn = pol.Polynomial.fit([0,25,50,70,100,120,160],[1210,980,900,800,200,188,0],3)
    bb3 = 0.9*np.maximum(fn(np.arange(240)),0)
    bb3[149:]=0

    new_shelf = nc.Dataset(filename, "w", format="NETCDF4")
    new_shelf.createDimension("ny", 40)
    new_shelf.createDimension("nx", 240)
    new_shelf.createVariable("thick","f8",("ny","nx"))
    new_shelf.createVariable("area","f8",("ny","nx"))

    for ii in range(40):
        new_shelf["area"][ii,:] = b5
        #bb4 = bb3 +(19.5-ii)**2*0.8 # previously 0.3
        bb4 = (bb3 +(19.5-ii)**2*0.8)*(1-(20-ii)**2/1000) 
        iip = abs(20-ii)
        for j in range(iip):
            bb4[164-j] = bb4[164-iip]*j/iip
        bb4[164:] = 0
        new_shelf["thick"][ii,:] = bb4

    new_shelf["thick"][:, 164:] = 0
    new_shelf["area"][:, 164:] = 0

    new_shelf.close()

In [6]:
#make_shelf(di + ICE_SHELF_FILE)

In [7]:
make_shelf_2(di + ICE_SHELF_FILE)

In [8]:
# make_thickness_file(di + THICKNESS_FILE)

In [ ]:
display_config()

### Plot output h,e,salt etc at chosen time

In [ ]:
prog = xr.open_dataset(di + '/'+PROG_FILE)
run_time = prog.coords["Time"].values[-1] - cftime.DatetimeJulian(1,1,1)
     
fig,axes = plt.subplots(nrows = 1, ncols = 2, figsize = (15,4))
tt=-1; zz= 12; yy=39; xx=64; ll = [0,zz,zz+1,-1] # ll=[0,1,2,3,4,5,10,-3,-2,-1]
for zzz in ll:
    axes[0].plot(prog.e[tt][zzz].data[yy,:])
    axes[1].plot(prog.e[tt][zzz].data[:,xx],'.')
axes[0].vlines(xx, -720, 0, linestyles='dashed')
axes[1].vlines(yy, -720, 0, linestyles='dashed')

fig.suptitle("Run time = {}".format(run_time))

In [ ]:
np.nanmin(prog.h[-1].data)

In [ ]:
### Can see that the initial salt is "bad" with THICKNESS_CONFIG="ISOMIP" but "fixed" with THICKNESS_CONFIG="thickness_file" hack
ttt = -1; zzz = 0
plt.plot(prog.salt.data[ttt][zzz][yy,:])

In [ ]:
print(np.nanmax(prog.u.data[-1]), np.nanmin(prog.u.data[-1])) # 0.0012509557073082142 -0.0006301157530793094

In [ ]:
print(np.nanmax(prog.v.data[-1]), np.nanmin(prog.v.data[-1])) # 0.0012509557073082142 -0.0006301157530793094

In [ ]:
arr = prog.u.data[-1]
ind = np.unravel_index(np.nanargmin(arr),arr.shape)
print(arr[ind])
ind

np.nanmin(prog.e.data[0])

In [ ]:
mo = xr.open_dataset(di + '/mom.nc')
fig,axes = plt.subplots(nrows = 5, ncols = 2, figsize = (12,7))

tt=-1; zz=11; yy=15; xx=16
showU = True
if (showU):
    u_or_v = "u"; PFu_or_v = "PFu"; prog_field = prog.u; mo_field = mo.PFu; 
    bt_PFu_or_v = "hor. friction accel. u"; mo_field2 = mo.diffu #mo.u_accel_bt #mo.du_dt_visc
else:
    u_or_v = "v"; PFu_or_v = "PFv"; prog_field = prog.v; mo_field = mo.PFv; 
    bt_PFu_or_v = "hor. friction accel. v"; mo_field2 = mo.diffv #mo.v_accel_bt #mo.dv_dt_visc
    
axes[0,0].plot(prog_field[0][zz].data[yy,:],'.')
axes[0,0].set_xlabel("x")
axes[0,0].set_title("{} at first output time".format(u_or_v)) # depends on diag_table setting
axes[0,1].plot(prog_field[0][zz].data[:,xx],'.')
axes[0,1].set_xlabel("y")
axes[0,1].set_title("{} at first output time".format(u_or_v)) # depends on diag_table setting
axes[1,0].plot(prog_field[tt][zz].data[yy,:],'.',)
axes[1,0].set_xlabel("x")
axes[1,0].set_title("{} at output time t = {}".format(u_or_v, tt))
axes[1,1].plot(prog_field[tt][zz].data[:,xx],'.')
axes[1,1].set_xlabel("y")
axes[1,1].set_title("{} at output time t = {}".format(u_or_v,tt))
axes[2,0].plot(mo_field[tt][zz].data[yy,:])
axes[2,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[2,0].set_xlabel("x")
axes[2,0].set_title("{} at output time t = {}".format(PFu_or_v,tt))
axes[2,1].plot(mo_field[tt][zz].data[:,xx])
axes[2,1].set_xlabel("y")
axes[2,1].set_title("{} at output time t = {}".format(PFu_or_v, tt))
axes[3,0].plot(mo_field2[tt][zz].data[yy,:])
#axes[3,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[3,0].set_xlabel("x")
axes[3,0].set_title("{} at output time t = {}".format(bt_PFu_or_v,tt))
axes[3,1].plot(mo_field2[tt][zz].data[:,xx])
axes[3,1].set_xlabel("y")
axes[3,1].set_title("{} at output time t = {}".format(bt_PFu_or_v, tt))
axes[4,0].plot(prog.h[tt][zz].data[yy,:],'.')
axes[4,0].set_xlabel("x")
axes[4,0].set_title("h at output time t = {}".format(tt))
axes[4,1].plot(prog.h[tt][zz].data[:,xx],'.',)
axes[4,1].set_xlabel("y")
axes[4,1].set_title("h at output time t = {}".format(tt))

plt.subplots_adjust(wspace = 0.25, hspace = 2.0)

mo.close()

In [ ]:
fc = xr.open_dataset(di + '/forcing.nc')

fig,axes = plt.subplots(nrows = 1, ncols = 2, figsize = (12,2))

tt=-1; yy=22; xx=15

#fc_field = fc.taux
#fc_field = fc.tauy
#fc_field = fc.ustar
#fc_field = fc.PRCmE
fc_field = fc.p_surf

#axes[0,0].plot(aft.u.data[zzz,yy+NHALO,NHALO:-NHALO])
axes[0].plot(fc_field[tt].data[yy,:],'.')
axes[0].set_xlabel("x")
axes[0].set_title("forcing at time t = {}".format(tt)) # depends on diag_table setting
axes[1].plot(fc_field[tt].data[:,xx],'.')
axes[1].set_xlabel("y")
axes[1].set_title("forcing at time t = {}".format(tt)) # depends on diag_table setting

In [ ]:
mo = xr.open_dataset(di + '/mom.nc')
fig,axes = plt.subplots(nrows =13, ncols = 2, figsize = (12,15))

tt=-1; zz=11; yy=15; xx=16
showU = True
if (showU):
    u_or_v = "u"; prog_field = prog.u;
    mo_field1_desc = "du/dt"; mo_field1 = mo.dudt
    mo_field2_desc = "PFu"; mo_field2 = mo.PFu  
    mo_field3_desc = "CAu"; mo_field3 = mo.CAu   
    mo_field4_desc = "u_accel_bt"; mo_field4 = mo.u_accel_bt  
    mo_field5_desc = "diffu"; mo_field5 = mo.diffu
    mo_field6_desc = "du/dt_visc"; mo_field6 = mo.du_dt_visc
else:
    u_or_v = "v"; prog_field = prog.v;
    mo_field1_desc = "dv/dt"; mo_field1 = mo.dvdt
    mo_field2_desc = "PFv"; mo_field2 = mo.PFv  
    mo_field3_desc = "CAv"; mo_field3 = mo.CAv   
    mo_field4_desc = "v_accel_bt"; mo_field4 = mo.v_accel_bt 
    mo_field5_desc = "diffv"; mo_field5 = mo.diffv
    mo_field6_desc = "dv/dt_visc"; mo_field6 = mo.dv_dt_visc
    
axes[0,0].plot(prog_field[0][zz].data[yy,:],'.')
axes[0,0].set_xlabel("x")
axes[0,0].set_title("{} at first output time".format(u_or_v)) # depends on diag_table setting
axes[0,1].plot(prog_field[0][zz].data[:,xx],'.')
axes[0,1].set_xlabel("y")
axes[0,1].set_title("{} at first output time".format(u_or_v)) # depends on diag_table setting
axes[1,0].plot(prog_field[tt][zz].data[yy,:],'.',)
axes[1,0].set_xlabel("x")
axes[1,0].set_title("{} at output time t = {}".format(u_or_v, tt))
axes[1,1].plot(prog_field[tt][zz].data[:,xx],'.')
axes[1,1].set_xlabel("y")
axes[1,1].set_title("{} at output time t = {}".format(u_or_v,tt))
axes[2,0].plot(mo_field1[tt][zz].data[yy,:])
axes[2,0].vlines(xx, -2e-10, 0, linestyles='dashed', colors='C2')
axes[2,0].set_xlabel("x")
axes[2,0].set_title("{} at output time t = {}".format(mo_field1_desc,tt))
axes[2,1].plot(mo_field1[tt][zz].data[:,xx])
axes[2,1].set_xlabel("y")
axes[2,1].set_title("{} at output time t = {}".format(mo_field1_desc, tt))
axes[3,0].plot(mo_field2[tt][zz].data[yy,:])
#axes[3,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[3,0].set_xlabel("x")
axes[3,0].set_title("{} at output time t = {}".format(mo_field2_desc,tt))
axes[3,1].plot(mo_field2[tt][zz].data[:,xx])
axes[3,1].set_xlabel("y")
axes[3,1].set_title("{} at output time t = {}".format(mo_field2_desc, tt))
axes[4,0].plot(mo_field3[tt][zz].data[yy,:])
#axes[4,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[4,0].set_xlabel("x")
axes[4,0].set_title("{} at output time t = {}".format(mo_field3_desc,tt))
axes[4,1].plot(mo_field3[tt][zz].data[:,xx])
axes[4,1].set_xlabel("y")
axes[4,1].set_title("{} at output time t = {}".format(mo_field3_desc, tt))
axes[5,0].plot(mo_field4[tt].data[yy,:])
#axes[5,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[5,0].set_xlabel("x")
axes[5,0].set_title("{} at output time t = {}".format(mo_field4_desc,tt))
axes[5,1].plot(mo_field4[tt].data[:,xx])
axes[5,1].set_xlabel("y")
axes[5,1].set_title("{} at output time t = {}".format(mo_field4_desc, tt))
axes[6,0].plot(mo_field5[tt][zz].data[yy,:])
#axes[6,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[6,0].set_xlabel("x")
axes[6,0].set_title("{} at output time t = {}".format(mo_field5_desc,tt))
axes[6,1].plot(mo_field5[tt][zz].data[:,xx])
axes[6,1].set_xlabel("y")
axes[6,1].set_title("{} at output time t = {}".format(mo_field5_desc, tt))
axes[7,0].plot(mo_field6[tt][zz].data[yy,:])
#axes[7,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[7,0].set_xlabel("x")
axes[7,0].set_title("{} at output time t = {}".format(mo_field6_desc,tt))
axes[7,1].plot(mo_field6[tt][zz].data[:,xx])
axes[7,1].set_xlabel("y")
axes[7,1].set_title("{} at output time t = {}".format(mo_field6_desc, tt))
axes[8,0].plot(mo_field2[tt][zz].data[yy,:]+mo_field6[tt][zz].data[yy,:])
#axes[8,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[8,0].set_xlabel("x")
axes[8,0].set_title("{} at t = {}".format("PF+d/dt_visc",tt))
axes[8,1].plot(mo_field2[tt][zz].data[:,xx]+mo_field6[tt][zz].data[:,xx])
axes[8,1].set_xlabel("y")
axes[8,1].set_title("{} at t = {}".format("PF+d/dt_visc", tt))
axes[9,0].plot(mo_field2[tt][zz].data[yy,:]+mo_field6[tt][zz].data[yy,:]
              +mo_field5[tt][zz].data[yy,:])
#axes[9,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[9,0].set_xlabel("x")
axes[9,0].set_title("{} at t = {}".format("PF+d/dt_visc+diff",tt))
axes[9,1].plot(mo_field2[tt][zz].data[:,xx]+mo_field6[tt][zz].data[:,xx]
              +mo_field5[tt][zz].data[:,xx])
axes[9,1].set_xlabel("y")
axes[9,1].set_title("{} at t = {}".format("PF+d/dt_visc+diff", tt))
axes[10,0].plot(mo_field2[tt][zz].data[yy,:]+mo_field6[tt][zz].data[yy,:]
              +mo_field5[tt][zz].data[yy,:]+mo_field4[tt].data[yy,:])
#axes[10,0].vlines(xx, 0, 0.2, linestyles='dashed', colors='C2')
axes[10,0].set_xlabel("x")
axes[10,0].set_title("{} at t = {}".format("PF+d/dt_visc+diff+accel_bt",tt))
axes[10,1].plot(mo_field2[tt][zz].data[:,xx]+mo_field6[tt][zz].data[:,xx]
              +mo_field5[tt][zz].data[:,xx]+mo_field4[tt].data[:,xx])
axes[10,1].set_xlabel("y")
axes[10,1].set_title("{} at t = {}".format("PF+d/dt_visc+diff+accel_bt", tt))
axes[11,0].plot(mo_field2[tt][zz].data[yy,:]+mo_field6[tt][zz].data[yy,:]
              +mo_field5[tt][zz].data[yy,:]+mo_field4[tt].data[yy,:]
               +mo_field3[tt][zz].data[yy,:])
axes[11,0].vlines(xx, -3.0e-10, 0, linestyles='dashed', colors='C2')
axes[11,0].set_xlabel("x")
axes[11,0].set_title("{} at t = {}".format("PF+d/dt_visc+diff+accel_bt+CA",tt))
axes[11,1].plot(mo_field2[tt][zz].data[:,xx]+mo_field6[tt][zz].data[:,xx]
              +mo_field5[tt][zz].data[:,xx]+mo_field4[tt].data[:,xx]
               +mo_field3[tt][zz].data[:,xx])
axes[11,1].set_xlabel("y")
axes[11,1].set_title("{} at t = {}".format("PF+d/dt_visc+diff+accel_bt+CA", tt))
axes[12,0].plot(prog.h[tt][zz].data[yy,:],'.')
axes[12,0].set_xlabel("x")
axes[12,0].set_title("h at t = {}".format(tt))
axes[12,1].plot(prog.h[tt][zz].data[:,xx],'.',)
axes[12,1].set_xlabel("y")
axes[12,1].set_title("h at output time t = {}".format(tt))

plt.subplots_adjust(wspace = 0.2, hspace = 2.2)

mo.close()

### Make an "adjusted salt" file

x=np.arange(240)
bb3=0.9*np.maximum(3e-7*(x-250)**4-17,0)

for ii in range(40):
    bb4 = bb3 +(19.5-ii)**2*0.8
    iip = abs(20-ii)
    for j in range(iip):
        bb4[164-j] = bb4[164-iip]*j/iip
        bb4[164:] = 0
    plt.plot(bb4)    

#! Open files saved by MOM6 needed to adjust the salt 
bt=xr.open_dataset(di + "/deb_bathyT.nc") # topography file needed to calculate e
h=xr.open_dataset(di + '/after_ts_2_h.nc') # h file before the first time step, but after MOM6 remap
S=xr.open_dataset(di + '/before_finish_MOM_initialization_S.nc') # salt file produced by MOM6, which we want to adjust 

#! calculate e and then s as a linear function of e 
S_TOP = 33.8
S_BOTTOM = 34.55
MAXIMUM_DEPTH = 720.0
e=h.h.sum(axis=0).data-bt.bathyT.data
s=(e-h.h.data.cumsum(axis=0) + h.h.data/2)*(S_BOTTOM-S_TOP)/(-MAXIMUM_DEPTH) + S_TOP

#! fix the halo points
s[:,:NHALO,:NHALO]=0
s[:,:NHALO,-NHALO:]=0
s[:,-NHALO:,:NHALO]=0
s[:,-NHALO:,-NHALO:]=0

#! create new netcdf4 file for the adjusted salt based on the original salt file
nz, ny, nx = S.S.data.shape
new_salt = nc.Dataset(di + "/INPUT/adjustedS.nc", "w", format="NETCDF4")
new_salt.createDimension("nz", nz)
new_salt.createDimension("ny", ny)
new_salt.createDimension("nx", nx)
temp = new_salt.createVariable("S","f8",("nz","ny","nx"))
temp[:,:,:]=s
new_salt.close()

In [ ]:
! cat MOM_override | tail

In [ ]:
prog.h[-1]

In [ ]:
fc = xr.open_dataset(di + '/forcing.nc')

In [ ]:
fc

In [ ]:
 p_surf_fix2=fc.p_surf[0,:,:]

In [ ]:
np.unique(fc.p_surf[0,:,:]-p_surf_fix2)

In [ ]:
ic=xr.open_dataset(di+"/"+IC_FILE)

In [ ]:
h_fix2=ic.h[0] # different

In [ ]:
eta_fix2=ic.eta[0] # same

In [ ]:
salt_fix2=ic.Salt[0] # different

In [ ]:
sfc_fix2=ic.sfc[0] # different

In [ ]:
plt.imshow(eta_fix2.data[0,:,:]-sfc_fix2.data[:,:])
plt.colorbar(shrink=0.5)

In [ ]:
plt.imshow((np.sum(h_fix2.data, axis=0)-eta_fix2[0].data-720))
plt.colorbar(shrink=0.5)

In [ ]:
plt.imshow((np.sum(h_fix2.data, axis=0)-sfc_fix2.data-720))
plt.colorbar(shrink=0.5)

In [ ]:
np.unique(ic.v2.data)

In [ ]:
## check initial salt is linear

ic=xr.open_dataset(di + "/" +IC_FILE)

# COLD profile
S_TOP = 33.8
S_BOTTOM = 34.55
#WARM profile
#S_TOP = 33.8
#S_BOTTOM = 34.7
MAXIMUM_DEPTH = 720.0

eta = ic.eta[0].data[0]
s=(eta - ic.h[0].data.cumsum(axis=0) + ic.h[0].data/2)*(S_BOTTOM-S_TOP)/(-MAXIMUM_DEPTH) + S_TOP

np.unique(ic.Salt[0].data-s)

In [ ]:
sfc=xr.open_dataset(di+"/sfc.nc") #Line 1408 MOM_dynamics_split_RK2.F90

In [ ]:
eta=xr.open_dataset(di+"/eta.nc") #line 3205 MOM.F90

In [ ]:
h=xr.open_dataset(di+"/h.nc") #Line 3218 MOM.F90

In [ ]:
h_split_init=xr.open_dataset(di+"/h_split_init.nc") 

In [ ]:
h_to_eta=xr.open_dataset(di+"/h_to_eta.nc") 

In [ ]:
bathy_split_init=xr.open_dataset(di+"/bathy_split_init.nc") 

In [ ]:
np.unique(h_split_init.h.data-h_to_eta.h.data)

In [ ]:
np.unique(h_split_init.h.data-h.h.data)

In [ ]:
np.unique(h.h.data[:,4:-4,4:-4]-h_fix2.data)

In [ ]:
np.unique(sfc.sfc.data[4:-4,4:-4]-sfc_fix2.data)

In [ ]:
np.unique(eta.eta.data[:,4:-4,4:-4]-eta_fix2.data)

In [ ]:
np.unique(bathy_split_init.bathy.data[4:-4,4:-4])

In [ ]:
print(h_split_init.h.data[:,4+20,-5])

In [ ]:
h_split_init.h.data[-2,4+20,-5]

In [ ]:
h = ic.h[0].data
e = np.zeros((26,40,240))
e[:,:,:]=-720.0
for i in range(240):
    for k in range(24,-1,-1):
        for j in range(40):
            e[k,j,i] = e[k+1,j,i]+h[k,j,i]

In [ ]:
np.unique(e[-1,:,:]-ic.eta[0].data[-1,:,:])

In [ ]:
plt.imshow((np.sum(h, axis=0)-e[0,:,:]-720))
plt.colorbar(shrink=0.5)

In [ ]:
et=ic.eta[0].data[0]
slice = (et[:,:]+720)/25
plt.plot(slice[0,:])

In [ ]:
slice[np.where(slice<1e-5)] = 1e-6

In [ ]:
h_new = np.zeros((25,40,240))

In [ ]:
for k in range(25):
    h_new[k,:,:] = slice

In [ ]:
plt.imshow(np.sum(h_new, axis=0)-et-720)
plt.colorbar(shrink=0.5)

In [ ]:
### new thickness file for THICKNESS_CONFIG="thickness_file"
### (depends on MOM6 output in IC_FILE, ie. use after a run)
def make_thickness_file_2(filename):
    
    ic = xr.open_dataset(di + '/' + IC_FILE)
    _, nz, ny, nx = ic.h.data.shape

    new_thick = nc.Dataset(filename, "w", format="NETCDF4")
    new_thick.createDimension("nz", nz)
    new_thick.createDimension("ny", ny)
    new_thick.createDimension("nx", nx)
    new_thick.createVariable("h","f8",("nz","ny","nx"))

    et=ic.eta[0].data[0]
    slice = (et[:,:]+720)/25
    slice[np.where(slice<1e-5)] = 1e-6
    h_new = np.zeros((25,40,240))
    for k in range(25):
        h_new[k,:,:] = slice

    new_thick["h"][:,:,:] = h_new
    new_thick.close()

In [ ]:
make_thickness_file_2(di + THICKNESS_FILE)

In [ ]:
ic2=xr.open_dataset(di + "/" + IC_FILE)

In [ ]:
plt.plot(ic2.h[0].data[0,0,:])

In [ ]:
x=np.arange(240)
#bb3=0.9*np.maximum(0.02*(x-250)**2-150,0) # max vel ~ 1e-5 after 5 days with 0.3 on line 23
bb3=0.9*np.maximum(3e-7*(x-250)**4-17,0)

In [ ]:
plt.plot(x,bb3)

In [ ]:
fn = pol.Polynomial.fit([0,25,50,70,100,120,160],[1210,980,900,800,200,188,0],3)

In [ ]:
bb3 = np.maximum(fn(np.arange(240)),0)
bb3[150:]=0
plt.plot(bb3)
plt.ylim([0,1200])

In [ ]:
for ii in range(40):
#for ii in range(3):
        bb4 = (bb3 +(19.5-ii)**2*0.8)*(1-(20-ii)**2/1000) # previously 0.3
        iip = abs(20-ii)
        for j in range(iip):
            bb4[150-j] = bb4[150-iip]*j/iip
        bb4[150:] = 0
        plt.plot(bb4)
        #new_shelf["thick"][ii,:] = bb4

In [ ]:
x=np.arange(240)
bb3=0.9*np.maximum(3e-7*(x-250)**4-17,0) # max vel ~ 1e-4 after 5 days with 0.8 on line 23


for ii in range(40):
    bb4 = bb3 +(19.5-ii)**2*0.8 # previously 0.3
    iip = abs(20-ii)
    for j in range(iip):
        bb4[164-j] = bb4[164-iip]*j/iip
    bb4[164:] = 0
    plt.plot(bb4)